# Рекомендация тарифов


Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже сделана.

Требуется построить модель с максимально большим значением accuracy (тнеобходимо довести долю правильных ответов по крайней мере до 0.75)

## Подготовка данных

Загрузим нужные для работы библиотеки и загрузим данные в переменную df

In [121]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

RANDOM_STATE_ = 12345

In [122]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [123]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Описание данных:**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

сalls — количество звонков <br>
minutes — суммарная длительность звонков в минутах <br>
messages — количество sms-сообщений <br>
mb_used — израсходованный интернет-трафик в Мб <br>
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [125]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [126]:
df.nunique()

calls        184
minutes     3144
messages     180
mb_used     3203
is_ultra       2
dtype: int64

Проверим соотношение классов - долю тарифов «Ультра» и «Смарт» в общем наборе данных:

In [127]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [128]:
print('Доля тарифа «Ультра»:', round((df[df['is_ultra'] == 1].shape[0] / df.shape[0]) * 100, 2), '%')
print('Доля тарифа «Смарт»:', round((df[df['is_ultra'] == 0].shape[0] / df.shape[0]) * 100, 2), '%')

Доля тарифа «Ультра»: 30.65 %
Доля тарифа «Смарт»: 69.35 %


Тариф "Смарт" преобладает в данных - это означает, что классы не сбалансированы с соотношением примерно равным 70/30

Итак, мы провели предварительный анализ данных, проверили данные на наличие пропусков, можно приступать к построению моделей и ответам на вопросы проекта

Для определения тарифа мы будем решать задачу классификации с учителем.

Но прежде имеет смысл смотреть на корреляцию признаков. При высокой их степени коррелированности возникает проблема мультиколлиниарность признаков. Проблема "мультиколлинеарности" - это и про переобучение модели, и про её интерпретируемость, и про сложность модели (содержание в модели избыточного количества переменных).

В чем негативный эффект сложной модели? Вот в чем: усложняется интерпретация параметров, оценки параметров ненадежны - получаются большие стандартные ошибки, которые меняются с изменением объема наблюдений, что делает модель непригодной для прогнозирования

In [129]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Мы видим, что параметры `calls` и `minutes` сильно коррелируют между собой - коэффициент корреляции стремится к единице. Имеет смысл убрать один из коррелирующих параметров для исключения мультиколлинеарности и переобучения модели.

## Подготовка данных к ML


Разобьём набор данных на тренировочную, валидационную и тестовую выборки в соотношении 3:1:1. Параметром RANDOM_STATE_ обеспечим воспроизводимость результатов разбиения выборки, чтобы при каждом запуске кода подвыборки были идентичными

In [130]:
df_train, test = train_test_split(df, train_size=0.6, random_state=RANDOM_STATE_)

df_valid, df_test = train_test_split(test, train_size=0.5, random_state=RANDOM_STATE_)

features_train = df_train.drop(['is_ultra', 'calls'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra', 'calls'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra', 'calls'], axis=1)
target_test = df_test['is_ultra']

print(df_train.shape[0] / df.shape[0])
print(df_valid.shape[0] / df.shape[0])
print(df_test.shape[0] / df.shape[0])

0.5998755444928439
0.2000622277535781
0.2000622277535781


In [131]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 3)
(1928,)
(643, 3)
(643,)
(643, 3)
(643,)


## ML


Использовать для ML будем модели LogisticRegression, DecisionTreeClassifier, RandomForestClassifier. Наша задача - подобрать лучшие гиперпараметры, максимизирующие метрику accuracy_score на валидационных данных. Выберем лучшую из моделей.

После того, как подберем гиперпараметры на валидации протестируем модели на тестовых данных. По результатам тестирования на тесте выберем модель, которую сможем передать в продакшн

Начнем с логистической регрессии

In [132]:
model = LogisticRegression(random_state=RANDOM_STATE_, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)

best_accuracy_valid_lr = accuracy_score(target_valid, predictions)
print('Accuracy на валидационной выборке:', best_accuracy_valid_lr)

Accuracy на валидационной выборке: 0.7076205287713841


Проверим качество модели решающего дерева

In [133]:
for i in range(1,20):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE_, max_depth=i)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print(f'max_depth = {i} :', accuracy)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7884914463452566
max_depth = 5 : 0.7884914463452566
max_depth = 6 : 0.7744945567651633
max_depth = 7 : 0.7713841368584758
max_depth = 8 : 0.7744945567651633
max_depth = 9 : 0.7791601866251944
max_depth = 10 : 0.7698289269051322
max_depth = 11 : 0.7667185069984448
max_depth = 12 : 0.7698289269051322
max_depth = 13 : 0.7573872472783826
max_depth = 14 : 0.7387247278382582
max_depth = 15 : 0.744945567651633
max_depth = 16 : 0.7387247278382582
max_depth = 17 : 0.7309486780715396
max_depth = 18 : 0.7216174183514774
max_depth = 19 : 0.7216174183514774


In [144]:
best_model = None
best_result = 0

for i in range(1,20):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE_, max_depth=i)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    if accuracy > best_accuracy_valid_dtc:
        i_ = i
        # сохраним наилучшую модель
        best_model_decision_tree = model.fit(features_train, target_train)
        #  сохраним наилучшее значение метрики accuracy на валидационных данных
        best_accuracy_valid_dtc = accuracy
print('Accuracy наилучшей модели на валидационной выборке:', best_accuracy_valid_dtc)
print('Глубина дерева:', i_)


Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566
Глубина дерева: 4


Проверим качество модели случайного леса

In [145]:
best_model = None
best_result = 0
for est in range(1, 50):
    # создадим модель с заданным количеством деревьев
    model = RandomForestClassifier(random_state=RANDOM_STATE_, n_estimators=est)
    # обучим модель на тренировочной выборке
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    # посчитаем качество модели на валидационной выборке
    result = accuracy_score(target_valid, predictions)
    if result > best_accuracy_valid_rfc:
        est_ = est
        # сохраним наилучшую модель
        best_model_random_forest = model.fit(features_train, target_train)
        #  сохраним наилучшее значение метрики accuracy на валидационных данных
        best_accuracy_valid_rfc = result

print('Accuracy наилучшей модели на валидационной выборке:', best_accuracy_valid_rfc)
print('Количество деревьев:', est_)

Accuracy наилучшей модели на валидационной выборке: 0.7822706065318819
Количество деревьев: 18


Соберём данные исследования в таблицу

In [146]:
data = [
     {'Accuracy': best_accuracy_valid_lr},
     {'Accuracy': best_accuracy_valid_dtc},
     {'Accuracy': best_accuracy_valid_rfc}
     ]
model_result = pd.DataFrame(data, index=['LogisticRegression',
                                         'DecisionTreeClassifier',
                                         'RandomForestClassifier'])
model_result

,Accuracy
LogisticRegression,0.707621
DecisionTreeClassifier,0.788491
RandomForestClassifier,0.782271


Значение Accuracy примерно равны у лучших моделей решающего дерева и случайного леса. Проверим их на тестовых данных

## Проверка модели на тестовой выборке

Проверим, как поведут себя лучшие модели на тестовых данных

Для повышения качества модели объединим валидационную и обучающую выборки

In [165]:
features_train_final = pd.concat([features_train, features_valid], axis=0, ignore_index=True)
target_train_final = pd.concat([target_train, target_valid], axis=0, ignore_index=True)

In [170]:
best_model_random_forest = RandomForestClassifier(random_state=RANDOM_STATE_, n_estimators=est_)
best_model_random_forest.fit(features_train_final, target_train_final)
predictions = best_model_random_forest.predict(features_test)
result = accuracy_score(target_test, predictions)

print(result)

0.7853810264385692


In [171]:
best_model_decision_tree = DecisionTreeClassifier(random_state=RANDOM_STATE_, max_depth=i_)
best_model_decision_tree.fit(features_train_final, target_train_final)
predictions = best_model_decision_tree.predict(features_test)
result = accuracy_score(target_test, predictions)

print(result)

0.7729393468118196


Мы видим, что на тестовых данных лучше повела себя модель случайного леса, превысив значение метрики качества, полученное на валидационных данных

Значение Accuracy далеко не всегда даёт корректные результаты в случае с неравномерными классами (в нашем случае).
Проверим наши модели также ещё на 2 метрики качества на тестовых данных

In [172]:
recall_score(target_test, best_model_random_forest.predict(features_test))

0.5221674876847291

In [173]:
recall_score(target_test, best_model_decision_tree.predict(features_test))

0.4039408866995074

In [174]:
precision_score(target_test, best_model_random_forest.predict(features_test))

0.7210884353741497

In [175]:
precision_score(target_test, best_model_decision_tree.predict(features_test))

0.7663551401869159

Мы видим, что по метрикам `Accuracy` и `Recall` лучшие результаты показывает модель случайного леса. По метрике `Precision` лучше результат у модели решающего дерева

## Проверка моделей на адекватность

In [178]:
confusion_matrix(target_valid, best_model_decision_tree.predict(features_valid))

array([[436,  18],
       [109,  80]])

In [179]:
confusion_matrix(target_test, best_model_decision_tree.predict(features_test))

array([[415,  25],
       [121,  82]])

In [180]:
confusion_matrix(target_valid, best_model_random_forest.predict(features_valid))

array([[452,   2],
       [  7, 182]])

In [181]:
confusion_matrix(target_test, best_model_random_forest.predict(features_test))

array([[399,  41],
       [ 97, 106]])

Используем простейшую (dummy) модель: DummyClassifier для нашей задачи бинарной классификации. DummyClassifier "предсказывает" наиболее часто встречающийся класс. Здесь мы получаем контрольную accuracy, чтобы сравнить её с результатом работы нашей самой лучшей модели. Наша лучшая модель должна "побить" DummyClassifier.
    
Второй вариант - сравнить самый часто встречающийся класс в наших данных (это is_ultra == 0). Таких значений 2229 в нашем датафрейме. Всего же значений в датафрейме 3214. Значит самый часто встречающийся класс "занимает" 69% (2229 / 3214 == 0.693528313627878). Так мы и получили контрольные данные для сравнительной оценки, построенной нами "лучшей" модели. Наша лучшая модель "бьёт" этот скор

In [182]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)

dummy = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)

print('Уникальные предсказанные метки: ', (np.unique(dummy_pred)))

print('Оценка на тестовых данных: ', accuracy_score(target_test, dummy_pred))

Уникальные предсказанные метки:  [0]
Оценка на тестовых данных:  0.6842923794712286


И лучшая модель решающего дерева и лучшая модель случайного леса "бьёт" показатель DummyClassifier и показатель наиболее часто встречающегося класса

**Выводы по работе**

Для решения задачи определения тарифа на имеющихся данных:
- исключен параметр количества звонков ввиду сильной корреляции с суммарной длительностью звонков в минутах
- было определено, что определять тариф будет задача классификации с учителем
- данные были разделены на тренировочную, валидационную и тестовую выборки в пропорции 60%:20%:20%
- на валидационной выборке по метрике `Accuracy` чуть лучше показала себя модель решающего дерева, на тестовых данных - модель случайного леса
- в целом по валидационным и тестовым данным мы видим, что по метрикам `Accuracy` и `Recall` лучшие результаты показывает модель случайного леса. По метрике `Precision` лучше результат у модели решающего дерева
- была проведена проверка на адекватность модели - получилось достичь плановых значений метрики качества выше 0.75

Таким образом, в работу берем модель `best_model_random_forest`